In [1]:
import json
import pandas as pd
import re

In [2]:
with open("/Users/nhatlan/Downloads/output.json", "r") as f:
    tasks = json.load(f)

In [3]:
def get_spans(task):
    annotation = task.get("annotations")[0].get('result')
    spans = []
    for anno in annotation:
        start = anno.get('value').get("start")
        end = anno.get('value').get("end")
        label = anno.get('value').get("labels") 
        if 'O' not in label:
            spans.append((start,end,label))
            
    return spans

In [4]:
def find_bio_in_text(text, spans):
    tokens_w_tags = []
    for match in re.finditer(r"\S+", text):
        token = match.group()
        start_tok = match.start()
        end_tok = match.end()
        
        tag = 'O'
        
        for start, end, label in spans:
            if start_tok >= start and end_tok <= end+1:
                if start_tok== start:
                    tag = f'B-{label[0]}'
                else: 
                    tag = f'I-{label[0]}'
                continue
        
        tokens_w_tags.append((token, tag))
        
    return tokens_w_tags
        
    

In [5]:
def convert_to_bio(tasks, output_path = None):
    with open(output_path, "w", encoding="utf-8") as out_f:
        for task in tasks:
            text = task.get("data", {}).get("text")
            if not text:
                continue

            spans = get_spans(task)
            tokens_with_tags = find_bio_in_text(text, spans)

            for token, tag in tokens_with_tags:
                out_f.write(f"{token} {tag}\n")
            out_f.write("\n")  
        
        

In [6]:
convert_to_bio(tasks, output_path= "output.txt")